In [1]:
import pandas as pd
import random
import datetime
from credentials import uri, user, pwd
from patent_neo4j.connection import Neo4jConnection
from functools import reduce

In [2]:
df = pd.read_csv("assignee_multiple_patents.csv")

In [3]:
Q1 = df['patent_counts'].quantile(0.25)
Q3 = df['patent_counts'].quantile(0.75)

In [4]:
random.seed(100)
df = df[(df["patent_counts"] < Q3)]

In [5]:
df.head()

,assignee_id,patent_counts
1,000139d6-82b6-4701-8cb7-999296d802fa,11
2,0002c937-3be3-498d-a07e-3b237bb9aed4,15
3,0002eaf9-03e6-4200-a404-719cc48d52e1,24
4,00033f59-25e5-446d-be66-f4b6dd32fbf5,25
7,0008828e-6ee6-4c43-b234-95c7cf7d48d2,10


In [6]:
assignee_list = list(df["assignee_id"])

In [7]:
conn = Neo4jConnection(uri, user, pwd)

In [8]:
ai = conn.query_assignee_patents(assignee_list=assignee_list)

In [9]:
ai.head()

,assignee_id,patent_id,patent_date,nber,num_citation
0,000139d6-82b6-4701-8cb7-999296d802fa,9435744,2016-09-06,None,1
1,000139d6-82b6-4701-8cb7-999296d802fa,8377713,2013-02-19,19,6
2,000139d6-82b6-4701-8cb7-999296d802fa,7633397,2009-12-15,21,1
3,000139d6-82b6-4701-8cb7-999296d802fa,7629885,2009-12-08,21,3
4,000139d6-82b6-4701-8cb7-999296d802fa,7545280,2009-06-09,21,1


In [10]:
ai = ai.dropna()

In [11]:
ai["nber_category"] = ai["nber"].apply(lambda x: x[0])

In [12]:
ai['patent_date'] = ai['patent_date'].astype(str).apply(datetime.datetime.strptime, args=("%Y-%m-%d",))

In [13]:
ai = ai.sort_values(by=["assignee_id", "patent_date"], ascending=[True, True]).reset_index()

In [14]:
ai.head()

,index,assignee_id,patent_id,patent_date,nber,num_citation,nber_category
0,5,000139d6-82b6-4701-8cb7-999296d802fa,7541926,2009-06-02,21,15,2
1,4,000139d6-82b6-4701-8cb7-999296d802fa,7545280,2009-06-09,21,1,2
2,3,000139d6-82b6-4701-8cb7-999296d802fa,7629885,2009-12-08,21,3,2
3,2,000139d6-82b6-4701-8cb7-999296d802fa,7633397,2009-12-15,21,1,2
4,1,000139d6-82b6-4701-8cb7-999296d802fa,8377713,2013-02-19,19,6,1


In [15]:
ai = ai.drop(["index"], axis=1)

In [16]:
ai.to_csv("raw_roots_norm.csv", index=False)

In [ ]:
assignee_set = set(ai["assignee_id"])

In [ ]:
ai["switch"] = 0

In [ ]:
ai.head()

In [ ]:
for a in assignee_set:
    indices = ai[ai["assignee_id"] == a].index
    initial_roots = {ai.iloc[indices[0],6]}
    num_switch = 0
    for i in indices[1:]:
        next_patent_root = ai.iloc[i,6]

        if next_patent_root not in initial_roots:
            num_switch = num_switch + 1
            
        ai.iloc[i,7] = num_switch
        initial_roots = initial_roots.union(next_patent_root) 

In [ ]:
ai[ai["assignee_id"] == list(assignee_set)[9]]

In [ ]:
ai.to_csv("switchers.csv", index=False)

In [ ]:
assignee_info = ai.groupby('assignee_id').agg(list).reset_index()

In [ ]:
assignee_info["roots"] = assignee_info["nber_category"].apply(set)
assignee_info["fine_roots"] = assignee_info["nber"].apply(set)

In [ ]:
assignee_info.tail(10)

In [ ]:
assignee_info["num_roots"] = assignee_info["roots"].apply(len)
assignee_info["num_fine_roots"] = assignee_info["fine_roots"].apply(len)
assignee_info["num_patents"] = assignee_info["patent_id"].apply(len)
assignee_info["total_citation"] = assignee_info["num_citation"].apply(sum)
assignee_info["avg_citation"] = assignee_info["total_citation"]/assignee_info["num_patents"]

In [ ]:
assignee_info.head(5)

In [ ]:
assignee_info[["num_roots", "num_fine_roots", "num_patents", "total_citation", "avg_citation"]].to_csv("roots_and_patents.csv", index=False)

In [ ]:
by_year

In [14]:
by_year = conn.query_patents_subnber_year(1976)
for i in range(1977, 2018):
    print(i)
    by_year = pd.concat([by_year, conn.query_patents_subnber_year(i)], ignore_index=True)

1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [16]:
by_year

,nber,count,year
0,None,1,1976
1,6,15872,1976
2,5,16716,1976
3,1,18432,1976
4,4,11135,1976
...,...,...,...
276,2,42687,2015
277,None,181393,2015
278,7,1,2015
279,None,304126,2016


In [15]:
by_year.to_csv("subnber_by_year.csv", index=False)